In [6]:
import pandas as pd
import boto3
import json
import psycopg2

In [7]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','key')
SECRET = config.get('AWS','secret')


In [8]:
s3 = boto3.resource('s3',
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                  )

bucket = s3.Bucket('udacity-dend')

#  Display data 
## Song Files

In [ ]:
for obj in bucket.objects.filter(Prefix='song_data'):
    print(obj)

## Log Files

In [ ]:
for obj in bucket.objects.filter(Prefix='log_data'):
    print(obj)

## Log Json file

In [4]:
content_object = s3.Object('udacity-dend', 'log_json_path.json')
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
print(file_content)
print(json_content)

{
    "jsonpaths": [
        "$['artist']",
        "$['auth']",
        "$['firstName']",
        "$['gender']",
        "$['itemInSession']",
        "$['lastName']",
        "$['length']",
        "$['level']",
        "$['location']",
        "$['method']",
        "$['page']",
        "$['registration']",
        "$['sessionId']",
        "$['song']",
        "$['status']",
        "$['ts']",
        "$['userAgent']",
        "$['userId']"
    ]
}
{'jsonpaths': ["$['artist']", "$['auth']", "$['firstName']", "$['gender']", "$['itemInSession']", "$['lastName']", "$['length']", "$['level']", "$['location']", "$['method']", "$['page']", "$['registration']", "$['sessionId']", "$['song']", "$['status']", "$['ts']", "$['userAgent']", "$['userId']"]}


# Display DB Tables

In [16]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

## Testing

In [13]:
print(
    """
    copy         staging_events 
    from         {}
    credentials  'aws_iam_role={}'
    region       'us-west-2'
    json         {};
    """.format(config['S3']['LOG_DATA'], 
               config['IAM']['IAM_ARN'], 
               config['S3']['LOG_JSONPATH'])
)


    copy         staging_events 
    from         's3://udacity-dend/log_data'
    credentials  'aws_iam_role=arn:aws:iam::567714418050:role/redRole'
    region       'us-west-2'
    json         's3://udacity-dend/log_json_path.json';
    


Query for Load errors

In [18]:
cur.execute("""
select query, substring(filename,22,25) as filename,line_number as line, 
substring(colname,0,12) as column, type, position as pos, substring(raw_line,0,30) as line_text,
substring(raw_field_value,0,15) as field_text, 
substring(err_reason,0,45) as reason
from stl_load_errors 
order by query desc
limit 10;
""")

In [20]:
try:
    cur.execute("""
select d.query, substring(d.filename,14,20), 
d.line_number as line, 
substring(d.value,1,16) as value,
substring(le.err_reason,1,48) as err_reason
from stl_loaderror_detail d, stl_load_errors le
where d.query = le.query
and d.query = pg_last_copy_id(); 
""")
except(e):
    print(e)